# Reading Signup Genius

In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


# %%
import requests 
import pandas as pd
import time 
from datetime import datetime
import numpy as np
import re

print("Reading Timetree website and analyzing data...")

# Reading in Signup Genius URL and data

intl_url = 'https://www.signupgenius.com/go/10c0f4aaeab2fa2fac07-july1'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(intl_url, headers=header)

# Finding the right table on the page

dfs = pd.read_html(r.text)
df = dfs[3]
df.rename(columns={0:'Role', 2:'Person'} , inplace = True)
df.drop(columns=1, inplace = True)

# %%
df.head(25)

Reading Timetree website and analyzing data...


,Role,Person
0,Sunday Greeter (3),All slots filled Jackie Barber Jac Cottam Ste...
1,Sunday Secretary,Randy Millar
2,Sunday Topic Bringer,Ricki Gage Asky Basky
3,Monday Greeter (3),2 of 3 slots filled Joshua Reeves Ravyn Stubb...
4,Monday Secretary,JoAnna Saunders
5,Monday Topic Bringer,Joshua Reeves
6,Tuesday Greeter (3),2 of 3 slots filled Joshua Reeves Jackie Barb...
7,Tuesday Secretary,Jackie Barber
8,Tuesday Topic Bringer,JoAnna Saunders
9,Wednesday Greeter (3),2 of 3 slots filled Ricki Gage JoAnna Saunder...


# Open Roles

Below service positions are still open!

In [2]:
 df[df['Person'] == 'Sign Up']



,Role,Person


In [3]:

to_remove = {
    '2 of 3 slots filled',
    '1 of 3 slots filled',
    'All slots filled',
    'Sign Up',
    'O yeah',
    'Asky Basky',
    'Has greeted before'
}

to_replace = {
    'Joshua Reeves Reeves' : 'Joshua Reeves',
    'Bill Collins Collins' : 'Bill Collins'
}

df['Person'] = df['Person'].str.replace('|'.join(to_remove),"", regex = True)
df['Person'] = df['Person'].str.replace('Joshua Reeves Reeves','Joshua Reeves')
df['Person'] = df['Person'].str.replace('Bill Collins Collins','Bill Collins')
df['Person'] = df['Person'].str.replace('Jesse J Gage','Jesse Gage')
df['Person'] = df['Person'].str.replace('Jesse J Gage','Jesse Gage')

df['Role'] = df['Role'].str.split('(').str[0]

# %%
# Adding emoji for title

emoji = []


for i in df['Role']: 
    if 'Greeter' in i:
        i = "\U0001F366 " # soft ice cream
    elif 'Topic' in i: 
        i = "\U0001F367 " # shaved ice
    elif 'Secretary' in i: 
        i = "\U0001F9C9 " # coconut
    emoji.append(i)

df['Emoji'] = emoji

# %%
# Adding verb for title

verb = []

for i in df['Role']: 
    if 'Greeter' in i:
        i = 'greet!'
    elif 'Topic' in i: 
        i = 'brings topic!'
    elif 'Secretary' in i: 
        i = 'Secretarys!'
    verb.append(i)

df['Verb'] = verb
df.head(40)

,Role,Person,Emoji,Verb
0,Sunday Greeter,Jackie Barber Jac Cottam Stephie Mathis,🍦,greet!
1,Sunday Secretary,Randy Millar,🧉,Secretarys!
2,Sunday Topic Bringer,Ricki Gage,🍧,brings topic!
3,Monday Greeter,Joshua Reeves Ravyn Stubbs,🍦,greet!
4,Monday Secretary,JoAnna Saunders,🧉,Secretarys!
5,Monday Topic Bringer,Joshua Reeves,🍧,brings topic!
6,Tuesday Greeter,Joshua Reeves Jackie Barber,🍦,greet!
7,Tuesday Secretary,Jackie Barber,🧉,Secretarys!
8,Tuesday Topic Bringer,JoAnna Saunders,🍧,brings topic!
9,Wednesday Greeter,Ricki Gage JoAnna Saunders,🍦,greet!


In [4]:

df['first'] = df['Person'].str.split().str[0]
df['second'] = df['Person'].str.split().str[2]
df['third'] = df['Person'].str.split().str[4]
df = df.fillna('')
df['Combined'] = df['first'] + " " + df['second'] + " " + df['third']
df['Count'] = df['Combined'].str.split().str.len()
df.head(50)

,Role,Person,Emoji,Verb,first,second,third,Combined,Count
0,Sunday Greeter,Jackie Barber Jac Cottam Stephie Mathis,🍦,greet!,Jackie,Jac,Stephie,Jackie Jac Stephie,3
1,Sunday Secretary,Randy Millar,🧉,Secretarys!,Randy,,,Randy,1
2,Sunday Topic Bringer,Ricki Gage,🍧,brings topic!,Ricki,,,Ricki,1
3,Monday Greeter,Joshua Reeves Ravyn Stubbs,🍦,greet!,Joshua,Ravyn,,Joshua Ravyn,2
4,Monday Secretary,JoAnna Saunders,🧉,Secretarys!,JoAnna,,,JoAnna,1
5,Monday Topic Bringer,Joshua Reeves,🍧,brings topic!,Joshua,,,Joshua,1
6,Tuesday Greeter,Joshua Reeves Jackie Barber,🍦,greet!,Joshua,Jackie,,Joshua Jackie,2
7,Tuesday Secretary,Jackie Barber,🧉,Secretarys!,Jackie,,,Jackie,1
8,Tuesday Topic Bringer,JoAnna Saunders,🍧,brings topic!,JoAnna,,,JoAnna,1
9,Wednesday Greeter,Ricki Gage JoAnna Saunders,🍦,greet!,Ricki,JoAnna,,Ricki JoAnna,2


In [5]:

df.loc[df['Count'] == 3, 'title'] = df['Emoji'] + df['first'] + ", " + df['second'] + " & " + df['third'] + " " + df['Verb']
df.loc[df['Count'] == 2, 'title'] = df['Emoji'] + df['first'] + " & " + df['second'] + " " + df['Verb']
df.loc[df['Count'] == 1, 'title'] = df['Emoji'] + df['first'] + " " +  df['Verb']

# %%
# Cleaning for export 

df['Day'] = df['Role'].str.split().str[0]
df['Role'] = df['Role'].str.split(" ", 1).str[1]
df.drop(columns={'Person','Verb','first','second','third','Combined','Count','Emoji'} , inplace = True)
df = df[['Day','Role','title']]
df.head()


# In[6]:


script_url = 'https://docs.google.com/document/d/1tVsWegpQ8LR7J20m8sRVA9z-XuKPzDwX7d8yQER-X1c/edit?usp=sharing'
zoom_url = 'https://us04web.zoom.us/j/406536344'
df.head()

url = []

for i in df['Role']: 
    if 'Greeter' in i:
        i = zoom_url
    elif 'Topic' in i: 
        i = zoom_url
    elif 'Secretary' in i: 
        i = script_url
    url.append(i)


df['url'] = url
df['all_day'] = 'false'
df['type'] = 'event'
df['description'] = ''


df = df.fillna('')
df.head(40)

,Day,Role,title,url,all_day,type,description
0,Sunday,Greeter,"🍦 Jackie, Jac & Stephie greet!",https://us04web.zoom.us/j/406536344,false,event,
1,Sunday,Secretary,🧉 Randy Secretarys!,https://docs.google.com/document/d/1tVsWegpQ8L...,false,event,
2,Sunday,Topic Bringer,🍧 Ricki brings topic!,https://us04web.zoom.us/j/406536344,false,event,
3,Monday,Greeter,🍦 Joshua & Ravyn greet!,https://us04web.zoom.us/j/406536344,false,event,
4,Monday,Secretary,🧉 JoAnna Secretarys!,https://docs.google.com/document/d/1tVsWegpQ8L...,false,event,
5,Monday,Topic Bringer,🍧 Joshua brings topic!,https://us04web.zoom.us/j/406536344,false,event,
6,Tuesday,Greeter,🍦 Joshua & Jackie greet!,https://us04web.zoom.us/j/406536344,false,event,
7,Tuesday,Secretary,🧉 Jackie Secretarys!,https://docs.google.com/document/d/1tVsWegpQ8L...,false,event,
8,Tuesday,Topic Bringer,🍧 JoAnna brings topic!,https://us04web.zoom.us/j/406536344,false,event,
9,Wednesday,Greeter,🍦 Ricki & JoAnna greet!,https://us04web.zoom.us/j/406536344,false,event,


# Adding Time to Frame

In [6]:
start_time = []

for i in df['Role']: 
    if 'Sec' in i:
        i =  'T02:15:00.0000Z'
    elif 'Topic' in i: 
        i = 'T02:30:00.0000Z'
    else: 
        i = 'T02:00:00.0000Z'
    start_time.append(i)

df['start_at'] = start_time

df['end_at'] = 'T03:30:00.0000Z'

# Export File to CSV for Load

In [7]:
# print("Data exporting...")
# df.to_csv(r'/Users/jaccottam/Desktop/Projects/INTL/db/timetree.csv')
# print("Data exported to db Check for file named timetree.csv.")

# Creating Date Table for Meeting Population

Use the following table to create dates for the meeting in the future.

In [8]:
def create_date_table(start='2022-08-01', end='2022-08-31'):
    df = pd.DataFrame({"Date": pd.date_range(start, end)})
    df["month"] = df.Date.dt.month
    df["day_of_Month"] = df.Date.dt.day
    df["year"] = df.Date.dt.year
    df["Day"] = df.Date.dt.day_name()
    return df

dates = create_date_table()

dates.head(50)

,Date,month,day_of_Month,year,Day
0,2022-08-01,8,1,2022,Monday
1,2022-08-02,8,2,2022,Tuesday
2,2022-08-03,8,3,2022,Wednesday
3,2022-08-04,8,4,2022,Thursday
4,2022-08-05,8,5,2022,Friday
5,2022-08-06,8,6,2022,Saturday
6,2022-08-07,8,7,2022,Sunday
7,2022-08-08,8,8,2022,Monday
8,2022-08-09,8,9,2022,Tuesday
9,2022-08-10,8,10,2022,Wednesday


In [9]:
# Input event timezones and category

df['start_timezone'] = 'America/New_York'
df['end_timezone'] = 'America/New_York'
df['category'] = 'schedule'

In [10]:
write_df = pd.merge(df,dates,how = 'inner')

write_df['Date'] = write_df['Date'].astype('string')


write_df['start_at'] = write_df['Date'] + write_df['start_at']
write_df['end_at'] = write_df['Date'] + write_df['end_at']

write_df.drop(columns = {'Day','Role','month','day_of_Month','year','Date'}, inplace = True)

# Example of the text that could be passed into the TimeTree API to write to a calendar

testing = write_df.iloc[1].to_dict()
testing

{'title': '🍦 Jackie, Jac & Stephie greet!',
 'url': 'https://us04web.zoom.us/j/406536344',
 'all_day': 'false',
 'type': 'event',
 'description': '',
 'start_at': '2022-08-14T02:00:00.0000Z',
 'end_at': '2022-08-14T03:30:00.0000Z',
 'start_timezone': 'America/New_York',
 'end_timezone': 'America/New_York',
 'category': 'schedule'}

# Writing One Record to Timetree

## Required 

- data:attributes:title	
- data:attributes:category	
- data:relationships:label:data:id
- data:relationships:label:data:type	

```
  '{
    "data": {
      "attributes": {
        "category": "schedule",
        "title": "Event title",
        "all_day": false,
        "start_at": "2019-03-18T09:00:00.000Z",
        "start_timezone": "UTC",
        "end_at": "2019-03-18T10:00:00.000Z",
        "end_timezone": "UTC",
        "description": "blah blah blah",
        "location": "Tokyo",
        "url": "https://example.com"
      },
      "relationships": {
        "label": {
          "data": {
            "id": "1234,1",
            "type": "label"
          }
        },
        "attendees": {
          "data": [
            { "id": "1234,12345", "type": "user" },
            { "id": "1234,56789", "type": "user" }
          ]
        }
      }
    }
  }'
```

```
j = (df.groupby(['ID','Location','Country','Latitude','Longitude'])
       .apply(lambda x: x[['timestamp','tide']].to_dict('records'))
       .reset_index()
       .rename(columns={0:'Tide-Data'})
       .to_json(orient='records'))
```

In [41]:
write_df['data'] = ''
write_df['attributes'] = ''
write_df[0:1].groupby(['data','title','category']).apply(lambda x: x[['start_at','end_at']].to_dict('records')).reset_index().to_json(orient = 'records', indent = 4)

'[\n    {\n        "data":"",\n        "title":"\\ud83c\\udf66 Jackie, Jac & Stephie greet!",\n        "category":"schedule",\n        "0":[\n            {\n                "start_at":"2022-08-07T02:00:00.0000Z",\n                "end_at":"2022-08-07T03:30:00.0000Z"\n            }\n        ]\n    }\n]'

# Ideas 

Use the defaultdict from collections - that seems like it could be promising and work out for nested dictionaries. You could then transfer the nested dictionary to json. 